In [1]:
import pandas as pd

In [72]:
fmpg_filename = 'input20190411R02.xlsx'
# 读入数据
fmct = pd.read_excel('FMCT.xlsx')
fmop = pd.read_excel('FMOP.xlsx')
fmvs = pd.read_excel('FMVS.xlsx')
frav = pd.read_excel('FRAV.xlsx')
fmpg = pd.read_excel(fmpg_filename, dtype=str).replace('nan', '')

In [73]:
# 重命名参考表列名
nfmvs = pd.DataFrame(fmvs[['VSAPPL', 'VSNUMDOC', 'VSTITRE']])
nfmct = pd.DataFrame(fmct[['CTAPPL', 'CTNUMDOC', 'CTTITRE']])
nfmop = pd.DataFrame(fmop[['OPAPPL', 'OPNUMDOC', 'OPTITRE']])
nfmvs.rename(columns={'VSAPPL': 'PROTOCOL', 'VSNUMDOC': 'REFERENCE', 'VSTITRE': 'TITLE'}, inplace=True)
nfmct.rename(columns={'CTAPPL': 'PROTOCOL', 'CTNUMDOC': 'REFERENCE', 'CTTITRE': 'TITLE'}, inplace=True)
nfmop.rename(columns={'OPAPPL': 'PROTOCOL', 'OPNUMDOC': 'REFERENCE', 'OPTITRE': 'TITLE'}, inplace=True)
docs = {'1': nfmvs, '2': nfmct, '3': nfmop}

In [74]:
# 日期格式化
def performace_dates_join(years, months, days):
    ll = []
    for y, m, d in zip(years, months, days):
        ll.append(y + m + d)
    return pd.Series(ll)

def deadline_dates_join(years, months, days):
    ll = []
    for y, m, d in zip(years, months, days):
        if y != '' and m != '' and d != '':
            ll.append(y + '-' + m + '-' + d)
        else:
            ll.append('')
    return pd.Series(ll)

In [75]:
frav = frav.applymap(lambda x: x.strip())
fmpg['AVOFFICIEL'] = fmpg.merge(frav[['AVNUMAP', 'AVOFFICIEL']], how='left', left_on='PGNUMAP', right_on='AVNUMAP')['AVOFFICIEL']

In [76]:
# 按导出列名重构fmpg
d = {'AC': fmpg['AVOFFICIEL'].str.strip(),
     'PROTOCOL': fmpg['PGAPPL'].str.strip(),
     'PGCODTE': fmpg['PGCODTE'],
     'REFERENCE': fmpg['PGNUMDOC'].str.strip(),
     'STATUS': fmpg['PGSIT'].str.strip(),
     'PERFORMACE_DOC': fmpg['PGAPPDOC1'].str.strip() + fmpg['PGAPPDOC2'].str.strip(),
     'PERFORMACE_DATE': performace_dates_join(fmpg['PGAPPA4'].str.strip(), fmpg['PGAPPMM'].str.strip(), fmpg['PGAPPJJ'].str.strip()),
     'FH_DEADLINE': fmpg['PGPRVH'],
     'CY_DEADLINE': fmpg['PGPRVC'],
     'DEADLINE_DOC': fmpg['PGPRVDOC1'].str.strip() + fmpg['PGPRVDOC2'].str.strip(),
     'DEADLINE_DATE': deadline_dates_join(fmpg['PGPRVA4'].str.strip(), fmpg['PGPRVMM'].str.strip(), fmpg['PGPRVJJ'].str.strip())
    }

nfmpg = pd.DataFrame(d, columns=['AC', 'PROTOCOL', 'PGCODTE', 'REFERENCE', 'STATUS', 'PERFORMACE_DOC', 'PERFORMACE_DATE', 
                              'FH_DEADLINE', 'CY_DEADLINE', 'DEADLINE_DOC', 'DEADLINE_DATE'])

In [77]:
# 根据doc类型分组
groupbycode = nfmpg.groupby(nfmpg['PGCODTE'])
dfs = {}
for k, v in groupbycode:
    dfs[k] = pd.DataFrame(v)

In [78]:
# 根据REFERENCE获取TITLE
mindex = ['AC', 'PROTOCOL', 'PGCODTE', 'REFERENCE', 'TITLE', 'STATUS', 'PERFORMACE_DOC', 'DEADLINE_DOC',
          'FH_DEADLINE', 'CY_DEADLINE', 'PERFORMACE_DATE', 'DEADLINE_DATE']

merged_dfs = {}
for k, v in dfs.items():
    merged_dfs[k] = pd.merge(v, docs[k], on=['PROTOCOL', 'REFERENCE'], how='left')[mindex]

In [69]:
filtered_df = merged_dfs['2']
filtered_df = filtered_df[filtered_df['PERFORMACE_DATE'].str.isalnum()]
filtered_df.to_excel('filtered_output_20190411.xlsx', columns=['AC', 'PROTOCOL', 'REFERENCE', 'TITLE', 'PERFORMACE_DOC', 'DEADLINE_DOC',
          'FH_DEADLINE', 'CY_DEADLINE', 'PERFORMACE_DATE', 'DEADLINE_DATE'])

In [68]:
not_filtered_df = merged_dfs['2']
not_filtered_df.to_excel('not_filtered_output_20190411.xlsx', columns=['AC', 'PROTOCOL', 'REFERENCE', 'TITLE', 'PERFORMACE_DOC', 'DEADLINE_DOC',
          'FH_DEADLINE', 'CY_DEADLINE', 'PERFORMACE_DATE', 'DEADLINE_DATE'])

In [79]:
model_output = merged_dfs['2']
model_output = model_output[model_output['PERFORMACE_DATE'].str.isalnum()]
model_output.to_excel('model_output_20190411.xlsx', columns=['PROTOCOL', 'REFERENCE', 'AC', 'PERFORMACE_DATE'])